What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [ ]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [ ]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/SWW postcodes.csv')

In [ ]:
SWW_postcode = SWW_postcode.set_index('postcode')

In [ ]:
SWW_postcode

In [ ]:
files = glob.glob(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/*')

In [ ]:
files

In [ ]:
labels = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv', header = None)

In [ ]:
labels

In [ ]:
labels = labels.set_index([0])

In [ ]:
labels

for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    postcode = postcode.set_index([0])
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.reindex(data.index.drop('Postcode'))
    SWW_postcode.merge(data, how='left', left_index=True, right_index=True)
    
    
    print('Processed {fname}'.format(fname=file))

In [ ]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ex.csv', header = None)

In [ ]:
postcode

In [ ]:
postcode = postcode.set_index([0])

In [ ]:
data = pd.concat([labels,postcode])

In [ ]:
data

In [ ]:
data.columns = data.iloc[0]

In [ ]:
data = data.reindex(data.index.drop('Postcode'))

In [ ]:
data

In [ ]:
SWW_postcode.merge(data, how='left', left_index=True, right_index=True)

In [ ]:
data.insert(0, 'date', 'NaN')

In [ ]:
data

In [ ]:
SWW_postcode[SWW_postcode.isnull()] = data